### Deep Learning Software
<br> cs231n Lecture 6

In [0]:
!pip3 install torch torchvision
import torch
import torchvision
import tensorflow as tf

### 원래 numpy로 코드짜면..

In [0]:
import numpy as np  #numpy는 cpu로만 가동
np.random.seed(0)

N,D = 3,4

x = np.random.randn(N,D)
y = np.random.randn(N,D)
z = np.random.randn(N,D)

a = x * y 
b = a + z
c = np.sum(b)

grad_c = 1.0
grad_b = grad_c * np.ones((N,D))
grad_a = grad_b.copy()
grad_z = grad_b.copy()
grad_x = grad_a * y
grad_y = grad_a * x

print(a, grad_a)
print(b, grad_b)
print(c, grad_c)

[[ 1.34251038  0.04868913  0.43442581  0.74772853]
 [ 2.79027931  0.20049663  0.297442    0.12927355]
 [ 0.26351668  0.26837482  0.12451648 -1.07931093]] [[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]
[[ 3.61226501 -1.40567654  0.48018432  0.56054468]
 [ 4.32305853  1.6698554   0.45238942  0.50743607]
 [-0.62426907 -1.71242165 -0.22339567 -0.92296196]] [[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]
6.717008537800067 1.0


In [0]:
##PyTorch Example
from torch.autograd import Variable
N, D = 3,4
x = Variable(torch.randn(N,D), requires_grad = True)
y = Variable(torch.randn(N,D), requires_grad = True)
z = Variable(torch.randn(N,D), requires_grad = True)

a = x*y
b = a+z
c = torch.sum(b)

c.backward()

# print(x.grad.data)
# print(y.grad.data)
# print(z.grad.data)

## PyTorch: Three Levels of Abstraction
**Tensor**: Imperative ndarray, but runs on GPU <br>
**Variable**: Node in computational graph; store data and gradient<br>
**Module**: A neural network layer; may store state or learnable weights


---



*   넘파이 와 비슷하지만 Tensor는 GPU 상에서 실행 가능하다.
*   신경망을 구성하고 학습하는 과정에서 자동으로 미분한다. 



In [0]:
dtype = torch.float
device = torch.device("cuda:0") #모든 텐서를 GPU로 보내는 코드
# device = torch.device("cpu")

N, D_in, H, D_out = 64, 1000, 100, 10

x = torch.randn(N, D_in).type(dtype)
y = torch.randn(N, D_out).type(dtype)
w1 = torch.randn(D_in, H).type(dtype)
w2 = torch.randn(H, D_out).type(dtype)

learning_rate = 1e-6
for t in range(500):
  h = x.mm(w1)
  h_relu = h.clamp(min=0)
  y_pred = h_relu.mm(w2)

  loss = (y_pred - y).pow(2).sum() #Mean Squared Error 
  if t% 100 == 99:
    print("{}번째 loss값: {}".format(t+1, loss))

  #역전파
  grad_y_pred = 2.0 * (y_pred - y)
  grad_w2 = h_relu.t().mm(grad_y_pred)
  grad_h_relu = grad_y_pred.mm(w2.t())
  grad_h = grad_h_relu.clone()
  grad_h[h<0] = 0  #softmax
  grad_w1 = x.t().mm(grad_h)

  w1 -= learning_rate * grad_w1
  w2 -= learning_rate * grad_w2

# print(w1, w2)

100번째 loss값: 619.2386474609375
200번째 loss값: 4.6539106369018555
300번째 loss값: 0.048646967858076096
400번째 loss값: 0.00084747897926718
500번째 loss값: 8.656668069306761e-05




---

**PyTorch Neural Network**
<br> **autograd**: 자동 미분하여 역전파 단계에서 연산 자동화 가능.


In [0]:
# 입력과 출력을 저장하기 위해 무작위 값을 갖는 Tensor를 생성
# requires_grad = False로 설정하여 역전파 중에 이 Tensor들에 대한 변화도를 계산할 필요가 없음을 나타냄. 
x = Variable(torch.randn(N, D_in)).to(device)  #to(device)는 cuda에 safely하게 인스턴스를 집어넣기 위함이다. 
y = Variable(torch.randn(N, D_out), requires_grad = False).to(device) #default가 false여서 안 써줘도 됨

w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6

for t in range(500):
  y_pred = x.mm(w1).clamp(min=0).mm(w2) #역전파 단계를 따로 구현할 필요가 없으므로 중간값들에 대한 참조를 하지 않는다

  loss = (y_pred - y).pow(2).sum() #loss는 (1,) 형태의 tensor
  if t % 100 == 99:
    print(t, loss.item())   #loss.item()은 loss의 스칼라 값이다.

  #autograd를 사용하여 역전파 함. 이는 requires_grad=True를 갖는 모든 Tensor에 대해 그래디언트 계산함. 
  loss.backward()   #w1.grad와 w2.grad는 w1과 w2에 대한 손실의 변화도를 갖는 Tensor가 된다. 

  #경사하강법을 사용해서 가중치를 수동으로 갱신함. 
  #torch.no_grad()로 감싸는 이유는 가중치들이 requires_grad=True지만 autograd에서는 이를 추적할 필요가 없기 때문. 
  #tensor.data가 tensor의 저장공간을 공유하긴 하지만, 이력을 추적하진 않는다. 
  with torch.no_grad():
    w1 -= learning_rate * w1.grad
    w2 -= learning_rate * w2.grad

    #가중치 갱신 후에는 수동으로 변화도를 0으로 만듬
    w1.grad.zero_()
    w2.grad.zero_()




99 413.3988342285156
199 1.6160252094268799
299 0.009349741041660309
399 0.00021846350864507258
499 3.935487620765343e-05


**사용자 정의 모델**


In [0]:
class MyReLU(torch.autograd.Function):
  '''
  torch.autograd.Function을 상속받아 사용자 정의 autograd Function을 구현하고, 
  Tensor 연산을 하는 순전파와 역전파 단계를 구현  
  '''
  @staticmethod
  def forward(ctx, input):
    """
    순전파 단계에서는 입력을 갖는 Tensor를 받아 출력을 갖는 Tensor로 반환시킴. 
    ctx는 컨텍스트 객체로 역전파 연산을 위한 정보 저장에 사용된다.
    ctx.save_for_backward 메소드를 사용하여 역전파 단계에서 사용할 어떠한 객체도 저장가능
    """
    ctx.save_for_backward(input)
    return input.clamp(min=0)

  @staticmethod
  def backward(ctx, grad_output):
      """ 
      역전파 단계에서는 출력에 대한 손실의 변화도를 갖는 Tensor를 받고, 입력에 대한 손실의 변화도를 계산함
      """
      input, = ctx.saved_tensors
      grad_input = grad_output.clone()
      grad_input[input < 0] = 0  # relu에 대한 derivate
      return grad_input



x = Variable(torch.randn(N, D_in)).to(device)  #to(device)는 cuda에 safely하게 인스턴스를 집어넣기 위함이다. 
y = Variable(torch.randn(N, D_out), requires_grad = False).to(device) #default가 false여서 안 써줘도 됨

w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6

for t in range(500):
  relu = MyReLU.apply

  y_pred = relu(x.mm(w1)).mm(w2)
  loss = (y_pred - y).pow(2).sum()

  if t % 100 == 99:
    print(t,loss.item())

  #autograd를 사용하여 역전파 함.
  loss.backward()   

  #경사하강법을 사용해서 가중치를 수동으로 갱신함. 
  #torch.no_grad()로 감싸는 이유는 가중치들이 requires_grad=True지만 autograd에서는 이를 추적할 필요가 없기 때문. 
  #tensor.data가 tensor의 저장공간을 공유하긴 하지만, 이력을 추적하진 않는다. 
  with torch.no_grad():
    w1 -= learning_rate * w1.grad
    w2 -= learning_rate * w2.grad

    #가중치 갱신 후에는 수동으로 변화도를 0으로 만듬
    w1.grad.zero_()
    w2.grad.zero_()


99 615.1189575195312
199 3.5873830318450928
299 0.03356011211872101
399 0.0005682189948856831
499 6.444560131058097e-05


**Neural Network**

In [0]:
# N은 배치 크기이며, D_in은 입력의 차원입니다;
# H는 은닉층의 차원이며, D_out은 출력 차원입니다.
N, D_in, H, D_out = 64, 1000, 100, 10

# 입력과 출력을 저장하기 위해 무작위 값을 갖는 Tensor를 생성합니다.
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# nn 패키지를 사용하여 모델을 순차적 계층(sequence of layers)으로 정의합니다.
# nn.Sequential은 다른 Module들을 포함하는 Module로, 그 Module들을 순차적으로
# 적용하여 출력을 생성합니다. 각각의 Linear Module은 선형 함수를 사용하여
# 입력으로부터 출력을 계산하고, 내부 Tensor에 가중치와 편향을 저장합니다.
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

# 또한 nn 패키지에는 널리 사용하는 손실 함수들에 대한 정의도 포함하고 있습니다;
# 여기에서는 평균 제곱 오차(MSE; Mean Squared Error)를 손실 함수로 사용하겠습니다.
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-4
for t in range(500):
    # 순전파 단계: 모델에 x를 전달하여 예상되는 y 값을 계산합니다. Module 객체는
    # __call__ 연산자를 덮어써(override) 함수처럼 호출할 수 있게 합니다.
    # 이렇게 함으로써 입력 데이터의 Tensor를 Module에 전달하여 출력 데이터의
    # Tensor를 생성합니다.
    y_pred = model(x)

    # 손실을 계산하고 출력합니다. 예측한 y와 정답인 y를 갖는 Tensor들을 전달하고,
    # 손실 함수는 손실 값을 갖는 Tensor를 반환합니다.
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # 역전파 단계를 실행하기 전에 변화도를 0으로 만듭니다.
    model.zero_grad()

    # 역전파 단계: 모델의 학습 가능한 모든 매개변수에 대해 손실의 변화도를
    # 계산합니다. 내부적으로 각 Module의 매개변수는 requires_grad=True 일 때
    # Tensor 내에 저장되므로, 이 호출은 모든 모델의 모든 학습 가능한 매개변수의
    # 변화도를 계산하게 됩니다.
    loss.backward()

    # 경사하강법(gradient descent)를 사용하여 가중치를 갱신합니다. 각 매개변수는
    # Tensor이므로 이전에 했던 것과 같이 변화도에 접근할 수 있습니다.
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

99 2.4191596508026123
199 0.02994726225733757
299 0.0007691961945965886
399 2.9122276828275062e-05
499 1.43080262660078e-06


**Optimization**

In [0]:
# N은 배치 크기이며, D_in은 입력의 차원입니다;
# H는 은닉층의 차원이며, D_out은 출력 차원입니다.
N, D_in, H, D_out = 64, 1000, 100, 10

# 입력과 출력을 저장하기 위해 무작위 값을 갖는 Tensor를 생성합니다.
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# nn 패키지를 사용하여 모델과 손실 함수를 정의합니다.
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)
loss_fn = torch.nn.MSELoss(reduction='sum')

# optim 패키지를 사용하여 모델의 가중치를 갱신할 Optimizer를 정의합니다.
# 여기서는 Adam을 사용하겠습니다; optim 패키지는 다른 다양한 최적화 알고리즘을
# 포함하고 있습니다. Adam 생성자의 첫번째 인자는 어떤 Tensor가 갱신되어야 하는지
# 알려줍니다.
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for t in range(500):
    # 순전파 단계: 모델에 x를 전달하여 예상되는 y 값을 계산합니다.
    y_pred = model(x)

    # 손실을 계산하고 출력합니다.
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # 역전파 단계 전에, Optimizer 객체를 사용하여 (모델의 학습 가능한 가중치인)
    # 갱신할 변수들에 대한 모든 변화도를 0으로 만듭니다. 이렇게 하는 이유는
    # 기본적으로 .backward()를 호출할 때마다 변화도가 버퍼(buffer)에 (덮어쓰지 않고)
    # 누적되기 때문입니다. 더 자세한 내용은 torch.autograd.backward에 대한 문서를
    # 참조하세요.
    optimizer.zero_grad()

    # 역전파 단계: 모델의 매개변수에 대한 손실의 변화도를 계산합니다.
    loss.backward()

    # Optimizer의 step 함수를 호출하면 매개변수가 갱신됩니다.
    optimizer.step()

99 40.203311920166016
199 0.5891818404197693
299 0.011365246027708054
399 0.00018329854356124997
499 1.130008513428038e-06


**제어 흐름 + 가중치 공유** <br>
은닉 계층을 갖는 완전히 연결 (full-connected) ReLU 신경망이 무작위로 0~3 사이 숫자를 선택하고, 가장 안쪽의 은닉층들을 계산하기 위해 동일한 가중치를 여러 번 재사용함. 